In [6]:
import pandas
import sys
from tqdm import tqdm
import pandas as pd

In [2]:
# import local modules
sys.path.append('../src')
from Fishnet import Fishnet
from ImageExporter import ImageExporter
from ImageCorrector import ImageCorrector
from ImageProcessor import ImageProcessor

# update files automatically
%load_ext autoreload
%autoreload 2
pandas.set_option('display.max_columns', 500)

/opt/homebrew/Caskroom/miniconda/base/envs/capstone/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
fc = Fishnet.load('../Gis/Fishnet/fishnet_quarter_mile_completed.pkl')

In [4]:
# Compute Urbanization Rate
for year in [2017, 2018, 2019, 2020, 2021, 2022]:
    fc.compute_difference(f'MeanPixel_{year}', f'MeanPixel_{year-1}', filtered=True, normalize = True)

fc.filtered_fishnet.rename(columns={'MeanPixel_2022-MeanPixel_2021': 'UrbanizationRate2022'}, inplace=True)
fc.filtered_fishnet.rename(columns={'MeanPixel_2021-MeanPixel_2020': 'UrbanizationRate2021'}, inplace=True)
fc.filtered_fishnet.rename(columns={'MeanPixel_2020-MeanPixel_2019': 'UrbanizationRate2020'}, inplace=True)
fc.filtered_fishnet.rename(columns={'MeanPixel_2019-MeanPixel_2018': 'UrbanizationRate2019'}, inplace=True)
fc.filtered_fishnet.rename(columns={'MeanPixel_2018-MeanPixel_2017': 'UrbanizationRate2018'}, inplace=True)
fc.filtered_fishnet.rename(columns={'MeanPixel_2017-MeanPixel_2016': 'UrbanizationRate2017'}, inplace=True)
fc.filtered_fishnet.drop(columns=['MeanPixel_2021-MeanPixel_2016'], inplace=True)

In [8]:
data = fc.filtered_fishnet
data.head(3)

,id,geometry,batch_id,neighbors,ImageCoordinates,Width,Height,MeanPixel_2016,Entropy_2016,MeanPixel_2017,Entropy_2017,MeanPixel_2018,Entropy_2018,MeanPixel_2019,Entropy_2019,MeanPixel_2020,Entropy_2020,MeanPixel_2021,Entropy_2021,MeanPixel_2022,Entropy_2022,UrbanizationRate2017,UrbanizationRate2018,UrbanizationRate2019,UrbanizationRate2020,UrbanizationRate2021,UrbanizationRate2022
6060154,6060154.0,"POLYGON ((-95.79985 29.79208, -95.79985 29.795...",23743.0,"[6056884, 6056885, 6056886, 6060153, 6060155, ...","(403, 524, 448, 564)",45.0,40.0,181.191667,0.868003,197.341667,0.771170,222.275000,0.552850,219.583333,0.581321,225.958333,0.511536,233.041667,0.423354,237.008333,0.367993,0.063333,0.097778,-0.010556,0.025000,0.027778,0.015556
6060155,6060155.0,"POLYGON ((-95.79583 29.79208, -95.79583 29.795...",23743.0,"[6056885, 6056886, 6056887, 6060154, 6060156, ...","(448, 524, 492, 564)",44.0,40.0,204.724432,0.716210,213.272727,0.642938,229.355114,0.470794,228.485795,0.481478,230.948864,0.450721,237.613636,0.359102,241.815341,0.293594,0.033523,0.063068,-0.003409,0.009659,0.026136,0.016477
6060156,6060156.0,"POLYGON ((-95.79181 29.79208, -95.79181 29.795...",23743.0,"[6056886, 6056887, 6056888, 6060155, 6060157, ...","(492, 524, 537, 564)",45.0,40.0,140.250000,0.992774,133.025000,0.998645,136.708333,0.996234,147.333333,0.982474,148.750000,0.979869,168.583333,0.923751,168.583333,0.923751,-0.028333,0.014444,0.041667,0.005556,0.077778,0.000000


In [10]:
additional_columns = pd.DataFrame(columns = [f"Neighbors_Delta{j}_{year}_{year+1}" for year in range(2017, 2022) for j in [1,2,3]])

for index, row in tqdm(data.iterrows(), total = len(data)):

  # filter only neighbors
  df_filtered = data[data["id"].isin(row["neighbors"])]

  # keep subset of Delta columns
  df_filtered = df_filtered[[f"UrbanizationRate{year}" for year in range(2017, 2023)]]

  # compute mean
  mean_neighbors = df_filtered.mean().tolist()

  # append to dataframe
  additional_columns.loc[len(additional_columns)] = mean_neighbors

# change index to prepare for merging
additional_columns.index = data.index

# add new features to the original dataframe
df = pd.concat([data, additional_columns], axis = 1)

  0%|          | 0/22581 [00:00<?, ?it/s]


KeyError: "None of [Index(['Delta1_2017_2018', 'Delta2_2017_2018', 'Delta3_2017_2018',\n       'Delta1_2018_2019', 'Delta2_2018_2019', 'Delta3_2018_2019',\n       'Delta1_2019_2020', 'Delta2_2019_2020', 'Delta3_2019_2020',\n       'Delta1_2020_2021', 'Delta2_2020_2021', 'Delta3_2020_2021',\n       'Delta1_2021_2022', 'Delta2_2021_2022', 'Delta3_2021_2022'],\n      dtype='object')] are in the [columns]"